## Reducing Commercial Aviation Fatalities

https://www.kaggle.com/c/reducing-commercial-aviation-fatalities/overview

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
import dask.dataframe as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal,stats,interpolate
from biosppy.signals import ecg,resp,eda,eeg
from multiprocessing import Pool
from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import log_loss,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

Function to reduce memory usage of a pandas dataframe by checking the range of values in each column

https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65

In [2]:
def reduce_mem_usage(df): 
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        
        if (col_type != object):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    return df

### Functions for feature engineering

In [3]:
def get_ecg_r_features(pilot_raw_data):
    # Function to generate heart rate and respiration rate features from 'ecg' and 'r' signals
    pilot = pilot_raw_data.iloc[0]['pilot']
    N = len(pilot_raw_data)
    FS = 256 # Sampling frequency
    new_pilot_data = pd.DataFrame()
    if 'id' in pilot_raw_data.columns:
        new_pilot_data['id'] = pilot_raw_data['id']
    else:
        new_pilot_data['id'] = [str(pilot) + "_" + str(i) for i in pilot_raw_data.index.values]
        
    new_pilot_data['crew'] = pilot_raw_data['crew']
    new_pilot_data['seat'] = pilot_raw_data['seat']
    new_pilot_data['time'] = pilot_raw_data['time']
    new_pilot_data['ecg'] = pilot_raw_data['ecg']
    new_pilot_data['r'] = pilot_raw_data['r']
    # It was found that some pilots have ecg value = 0, maybe due to some problem in measuring ecg values
    # We are imputing new feature values for such pilots with median values for those features
    if all(v==0 for v in pilot_raw_data['ecg'].values):
        new_pilot_data['heart_rate'] = [62.730858] * N
        new_pilot_data['heart_rate_diff'] = [-1.8538252e-05] * N
        new_pilot_data['resp_rate'] = [0.25485292] * N
        new_pilot_data['resp_rate_diff'] = [-2.862812e-07] * N
    else:
        # https://biosppy.readthedocs.io/en/stable/biosppy.signals.html#biosppy-signals-ecg
        ts,filtered,rpeaks,templates_ts,templates,heart_rate_ts,heart_rate = ecg.ecg(signal=pilot_raw_data['ecg'].values, sampling_rate=FS, show=False)
        #The heart rate will be given as samples at different points from the above line. We can extrapolate these samples to fit the whole data
        f = interpolate.interp1d(heart_rate_ts, heart_rate, kind='cubic', fill_value='extrapolate')
        new_pilot_data['heart_rate'] = f(pilot_raw_data['time'].values)
        new_pilot_data['heart_rate_diff'] = new_pilot_data['heart_rate'].diff().fillna(0)
        
        #https://biosppy.readthedocs.io/en/stable/biosppy.signals.html#biosppy-signals-resp
        ts, filtered, zeros, resp_rate_ts, resp_rate_samples = resp.resp(signal=pilot_raw_data['r'].values, sampling_rate=FS, show=False)
        #The respiration rate will be given as samples at different points from the above line. We can inpolate these samples to fit the whole data
        f = interpolate.interp1d(resp_rate_ts, resp_rate_samples, kind='cubic', fill_value='extrapolate')
        new_pilot_data['resp_rate'] = f(pilot_raw_data['time'].values)
        new_pilot_data['resp_rate_diff'] = new_pilot_data['resp_rate'].diff().fillna(0)
    return new_pilot_data

In [4]:
def get_gsr_features(pilot_raw_data):
    # Function to generate amplitude, peaks and other features from 'gsr' signal
    pilot = pilot_raw_data.iloc[0]['pilot']
    N = len(pilot_raw_data)
    FS = 256 # Sampling frequency
    new_pilot_data = pd.DataFrame()
    if 'id' in pilot_raw_data.columns:
        new_pilot_data['id'] = pilot_raw_data['id']
    else:
        new_pilot_data['id'] = [str(pilot) + "_" + str(i) for i in pilot_raw_data.index.values]
    
    new_pilot_data['gsr'] = pilot_raw_data['gsr']
    # It was found that some pilots have gsr value = 0 for the entire duration, maybe due to some problem in measuring gsr values
    # We are imputing new feature values for such pilots with zero for those features
    if all(v==0 for v in pilot_raw_data['gsr'].values):
        new_pilot_data['gsr_diff'] = [0] * N
        new_pilot_data['gsr_diff_2'] = [0] * N
        new_pilot_data['gsr_last_onset'] = [N/FS] * N
        new_pilot_data['gsr_last_peak'] = [N/FS] * N
        new_pilot_data['gsr_amplitude'] = [0] * N
    else:
        new_pilot_data['gsr_diff'] = new_pilot_data['gsr'].diff().fillna(0)
        new_pilot_data['gsr_diff_2'] = new_pilot_data['gsr_diff'].diff().fillna(0)
        ts,filtered,onsets,peaks,amplitudes = eda.eda(signal=pilot_raw_data['gsr'].values, sampling_rate=FS, show=False)
        last_onset = [N] * N
        if len(onsets) > 0:
            for i in range(1,len(onsets)):
                last_onset[onsets[i-1]:onsets[i]] = list(range(0,(onsets[i]-onsets[i-1])))
            last_onset[onsets[-1]:] = list(range(0,(N-onsets[-1])))
        last_onset = [i/FS for i in last_onset]
        new_pilot_data['gsr_last_onset'] = last_onset
        
        last_peak = [N] * N
        if len(peaks) > 0:
            for i in range(1,len(peaks)):
                last_peak[peaks[i-1]:peaks[i]] = list(range(0,(peaks[i]-peaks[i-1])))
            last_peak[peaks[-1]:] = list(range(0,(N-peaks[-1])))
        last_peak = [i/FS for i in last_peak]
        new_pilot_data['gsr_last_peak'] = last_peak
        
        if len(onsets) < 4:
            new_pilot_data['gsr_amplitude'] = [0] * N
        else:
            f_amp = interpolate.interp1d(onsets, amplitudes, kind='cubic', fill_value='extrapolate')
            new_pilot_data['gsr_amplitude'] = f_amp(pilot_raw_data['time'].values)
    return new_pilot_data

In [5]:
def get_eeg_features(pilot_raw_data):
    # Function to generate power features from 'eeg' signals
    pilot = pilot_raw_data.iloc[0]['pilot']
    N = len(pilot_raw_data)
    FS = 256 # Sampling frequency
    EEG_SIGNAL_NAMES = ['eeg_fp1', 'eeg_f7', 'eeg_f8', 'eeg_t4', 'eeg_t6', 'eeg_t5', 'eeg_t3', 'eeg_fp2', 
                    'eeg_o1', 'eeg_p3', 'eeg_pz', 'eeg_f3', 'eeg_fz', 'eeg_f4', 'eeg_c4', 'eeg_p4', 
                    'eeg_poz', 'eeg_c3', 'eeg_cz', 'eeg_o2']
    new_pilot_data = pd.DataFrame()
    if 'id' in pilot_raw_data.columns:
        new_pilot_data['id'] = pilot_raw_data['id']
    else:
        new_pilot_data['id'] = [str(pilot) + "_" + str(i) for i in pilot_raw_data.index.values]
    
    eeg_signals = pilot_raw_data[EEG_SIGNAL_NAMES].values
    # https://biosppy.readthedocs.io/en/stable/biosppy.signals.html#biosppy-signals-eeg
    ts,theta,alpha_low,alpha_high,beta,gamma = eeg.get_power_features(signal=eeg_signals, sampling_rate=FS, size=40, overlap=0.99375)
    for i in range(len(EEG_SIGNAL_NAMES)):
        f_theta = interpolate.interp1d(ts, theta[:,i], kind='cubic', fill_value='extrapolate')
        new_pilot_data[EEG_SIGNAL_NAMES[i]+"_theta"] = f_theta(pilot_raw_data['time'].values)
        f_alpha_low = interpolate.interp1d(ts, alpha_low[:,i], kind='cubic', fill_value='extrapolate')
        new_pilot_data[EEG_SIGNAL_NAMES[i]+"_alpha_low"] = f_alpha_low(pilot_raw_data['time'].values)
        f_alpha_high = interpolate.interp1d(ts, alpha_high[:,i], kind='cubic', fill_value='extrapolate')
        new_pilot_data[EEG_SIGNAL_NAMES[i]+"_alpha_high"] = f_alpha_high(pilot_raw_data['time'].values)
        f_beta = interpolate.interp1d(ts, beta[:,i], kind='cubic', fill_value='extrapolate')
        new_pilot_data[EEG_SIGNAL_NAMES[i]+"_beta"] = f_beta(pilot_raw_data['time'].values)
        f_gamma = interpolate.interp1d(ts, gamma[:,i], kind='cubic', fill_value='extrapolate')
        new_pilot_data[EEG_SIGNAL_NAMES[i]+"_gamma"] = f_gamma(pilot_raw_data['time'].values)

    if 'event' in pilot_raw_data.columns:
        new_pilot_data['event'] = pilot_raw_data['event']
    return new_pilot_data

In [6]:
def get_new_features(pilot_raw_data):
    # Function to combine all the new features
    ecg_r_features = get_ecg_r_features(pilot_raw_data)
    gsr_features = get_gsr_features(pilot_raw_data)
    eeg_features = get_eeg_features(pilot_raw_data)
    
    pilot_new_features = pd.merge(ecg_r_features, gsr_features, on='id')
    pilot_new_features = pd.merge(pilot_new_features, eeg_features, on='id')
    pilot_new_features = reduce_mem_usage(pilot_new_features)
    return pilot_new_features

## Prediction

Function to return final predictions on the given raw input data

In [7]:
def get_predictions(X):
    # Preprocessing
    exp_mapping = {'CA':0, 'DA':1, 'SS':2, 'LOFT':3}
    X['experiment'] = X['experiment'].map(exp_mapping)
    X['pilot'] = X['crew'] * 100 + X['experiment'] * 10 + X['seat']
    pilots = [pilot for pilot in X['pilot'].unique()]
    
    # Feature engineering and prediction
    predictions = []
    model = pickle.load(open('rf_model.sav', 'rb')) # Random Forest model which has been selected for final predictions
    crew_ohe = pickle.load(open('crew_ohe.sav','rb')) # One hot encoder for crew id, which has already been trained
    crew_ohe_features = ["crew_"+str(i) for i in crew_ohe.categories_[0]]
    for pilot in tqdm(pilots):
        # Check if new features are already generated and stored in a file, else generate them
        if os.path.isfile("new_features/"+str(pilot)+".gzip"):
            pilot_new_features = pd.read_parquet("new_features/"+str(pilot)+".gzip")
        else:
            pilot_raw_data = X.loc[X['pilot'] == pilot]
            pilot_raw_data = pilot_raw_data.iloc[pilot_raw_data['time'].argsort()] # Sorting data according to time for each pilot
            pilot_raw_data.reset_index(drop=True, inplace=True)
            pilot_new_features = get_new_features(pilot_raw_data)
            pilot_new_features[crew_ohe_features] = crew_ohe.transform(pilot_new_features[['crew']]).toarray()
            for feature in crew_ohe_features:
                pilot_new_features[feature] = pilot_new_features[feature].astype(np.int8)
            pilot_new_features.drop(['crew'], axis=1, inplace=True)
        
        df_pred = pd.DataFrame()
        df_pred['id'] = pilot_new_features['id']
        pilot_new_features = pilot_new_features.drop(['id'], axis=1)
        y_pred = model.predict_proba(pilot_new_features)
        df_pred['A'] = y_pred[:,0]
        df_pred['B'] = y_pred[:,3]
        df_pred['C'] = y_pred[:,1]
        df_pred['D'] = y_pred[:,2]
        predictions.append(df_pred)
    predictions = pd.concat(predictions, ignore_index=True)
    return predictions

In [8]:
X_test = pd.read_csv('test.csv')
predictions = get_predictions(X_test)
predictions.to_csv("final_submission.csv", index=False)
!kaggle competitions submit -c reducing-commercial-aviation-fatalities -f final_submission.csv -m "This output was generated by Random Forest model which was selected for final submission"

100%|██████████| 18/18 [09:55<00:00, 33.10s/it]


100%|████████████████████████████████████████| 585M/585M [00:07<00:00, 85.5MB/s]
Successfully submitted to Reducing Commercial Aviation Fatalities

<img src='https://i.imgur.com/NKif5oi.png'>

## Scoring 

Function to return log loss score on given raw input data and target values

In [9]:
def get_score(X,y):
    X['event'] = y
    # Preprocessing
    event_mapping = {'A': 0, 'C': 1, 'D': 2, 'B': 3}
    X['event'] = X['event'].map(event_mapping)
    exp_mapping = {'CA':0, 'DA':1, 'SS':2, 'LOFT':3}
    X['experiment'] = X['experiment'].map(exp_mapping)
    X['pilot'] = X['crew'] * 100 + X['experiment'] * 10 + X['seat']
    pilots = [pilot for pilot in X['pilot'].unique()]
    
    # Feature engineering and prediction
    predictions = []
    target_values = []
    model = pickle.load(open('rf_model.sav', 'rb')) # Random Forest model which has been selected for final predictions
    crew_ohe = pickle.load(open('crew_ohe.sav','rb')) # One hot encoder for crew id, which has already been trained
    crew_ohe_features = ["crew_"+str(i) for i in crew_ohe.categories_[0]]
    for pilot in tqdm(pilots):
        # Check if new features are already generated and stored in a file, else generate them
        if os.path.isfile("new_features/"+str(pilot)+".gzip"):
            pilot_new_features = pd.read_parquet("new_features/"+str(pilot)+".gzip")
        else:
            pilot_raw_data = X.loc[X['pilot'] == pilot]
            pilot_raw_data = pilot_raw_data.iloc[pilot_raw_data['time'].argsort()] # Sorting data according to time for each pilot
            pilot_raw_data.reset_index(drop=True, inplace=True)
            pilot_new_features = get_new_features(pilot_raw_data)
            pilot_new_features[crew_ohe_features] = crew_ohe.transform(pilot_new_features[['crew']]).toarray()
            for feature in crew_ohe_features:
                pilot_new_features[feature] = pilot_new_features[feature].astype(np.int8)
            pilot_new_features.drop(['crew'], axis=1, inplace=True)
            pilot_new_features.to_parquet("new_features/"+str(pilot)+".gzip",compression='gzip')
            
        y = pilot_new_features['event']
        pilot_new_features = pilot_new_features.drop(['id','event'], axis=1)
        y_pred = model.predict_proba(pilot_new_features)
        predictions.extend(y_pred)
        target_values.extend(y)
    loss = log_loss(target_values, predictions)
    return loss

In [10]:
df_train = pd.read_csv('train.csv')
y = df_train['event']
X = df_train.drop(['event'], axis=1)
loss = get_score(X,y)
print('Multi class log loss for the given input and target values is ',loss)

100%|██████████| 54/54 [04:19<00:00,  4.81s/it]


Multi class log loss for the given input and target values is  0.00035128547908662216
